In [ ]:
%load_ext watermark
%watermark -a 'Christian Schuhegger' -u -d -v -p numpy,pandas,matplotlib,seaborn,rpy2,libpgm,pgmpy,networkx,graphviz,xarray,dsbasics,pytest
#,pygraphviz

In [ ]:
%matplotlib inline
import numpy as np, scipy, scipy.stats as stats, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pgmpy, pgmpy.models, pgmpy.factors.discrete, pgmpy.inference, libpgm, pytest
import datetime, time
import matplotlib.dates
import pytz
from dateutil import relativedelta
import timeit

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

sns.set()

In [ ]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

#HTML('<style>{}</style>'.format(CSS))

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport dsbasics.bn

* [rpy2](https://rpy2.bitbucket.io/) on bitbucket
* [rpy2](https://rpy2.github.io/) on github pages

* [bnlearn](http://www.bnlearn.com/)
* [manual](http://www.bnlearn.com/documentation/man/)

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'C')

import rpy2, rpy2.rinterface, rpy2.robjects, rpy2.robjects.packages, rpy2.robjects.lib, rpy2.robjects.lib.grid, \
    rpy2.robjects.lib.ggplot2, rpy2.robjects.pandas2ri, rpy2.interactive.process_revents, \
    rpy2.interactive, rpy2.robjects.lib.grdevices
# rpy2.interactive.process_revents.start()
rpy2.robjects.pandas2ri.activate()

# import R's "base" package
base = rpy2.robjects.packages.importr('base')
# import R's utility package
utils = rpy2.robjects.packages.importr('utils')
# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

# R package names
packnames = ('bnlearn', 'gRain')

# R vector of strings

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpy2.robjects.packages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(rpy2.robjects.StrVector(names_to_install))

grdevices   = rpy2.robjects.packages.importr('grDevices')
bnlearn     = rpy2.robjects.packages.importr('bnlearn')
gRain       = rpy2.robjects.packages.importr('gRain')

The test case follows exercise 5.1 from "Doing Bayesian Data Analysis" by John K. Kruschke:
* https://sites.google.com/site/doingbayesiandataanalysis/exercises
* [Kruschke-DBDA2E-ExerciseSolutions.pdf](https://sites.google.com/site/doingbayesiandataanalysis/exercises/Kruschke-DBDA2E-ExerciseSolutions.pdf) p. 17

In [ ]:
p_disease_present = 0.001
prior = pd.DataFrame([p_disease_present, 1-p_disease_present], columns=['disease_state'])
prior.index = pd.Index(['disease_present', 'disease_absent'])

p_test_positive_given_disease_present = 0.99
p_test_positive_given_disease_absent = 0.05

disease_test_cpd_df = pd.DataFrame([[p_test_positive_given_disease_present, p_test_positive_given_disease_absent],
                                    [1 - p_test_positive_given_disease_present, 1 - p_test_positive_given_disease_absent]], columns=['disease_present', 'disease_absent'])
disease_test_cpd_df.index = pd.Index(['test_positive', 'test_negative'])

posterior = prior['disease_state'].copy()

# selected_row = disease_test_cpd_df.loc['test-positive', :]
# posterior = selected_row * posterior
# posterior = posterior / float(posterior.sum())
# posterior['disease-present']

#for x in ['test_positive', 'test_negative']:
for x in ['test_positive']:
    selected_row = disease_test_cpd_df.loc[x,:]
    posterior = selected_row * posterior
    posterior = posterior / float(posterior.sum())


In [ ]:
prior

In [ ]:
disease_test_cpd_df

In [ ]:
posterior

In [ ]:
disease_state_CPD = pgmpy.factors.discrete.TabularCPD(variable='disease_state',
                                                      variable_card=2,
                                                      values=[[p_disease_present], [1.0 - p_disease_present]])
print(disease_state_CPD)

In [ ]:
test_result_CPD_1 = pgmpy.factors.discrete.TabularCPD(
     variable='test_result1',
     variable_card=2,
     values=[[p_test_positive_given_disease_present, p_test_positive_given_disease_absent],
             [(1 - p_test_positive_given_disease_present), (1 - p_test_positive_given_disease_absent)]],
     evidence=['disease_state'],
     evidence_card=[2])
print(test_result_CPD_1)

In [ ]:
test_result_CPD_2 = pgmpy.factors.discrete.TabularCPD(
     variable='test_result2',
     variable_card=2,
     values=[[p_test_positive_given_disease_present, p_test_positive_given_disease_absent],
             [(1 - p_test_positive_given_disease_present), (1 - p_test_positive_given_disease_absent)]],
     evidence=['disease_state'],
     evidence_card=[2])
print(test_result_CPD_2)

In [ ]:
model = pgmpy.models.BayesianModel()

model.add_nodes_from(['disease_state', 'test_result1', 'test_result2'])
model.add_edge('disease_state', 'test_result1')
model.add_edge('disease_state', 'test_result2')

model.add_cpds(disease_state_CPD, test_result_CPD_1, test_result_CPD_2)
model.check_model()

* [TypeError: object of type 'dict_keyiterator' has no len()](https://github.com/pgmpy/pgmpy/issues/927)

In [ ]:
# infr1 = pgmpy.inference.BeliefPropagation(model)
# evidence = {'test_result1': 0}
# query_vars = ['disease_state']
# p_disease = infr1.query(variables=query_vars, evidence=evidence)['disease_state'].values[0]
# # pprint('{0:f}'.format(p_disease))
# assert p_disease == pytest.approx(0.01943463, 0.0000001)

In [ ]:
# p_disease

In [ ]:
# infr2 = pgmpy.inference.VariableElimination(model)
# p_disease = infr2.query(variables=query_vars, evidence=evidence)['disease_state'].values[0]
# assert p_disease == pytest.approx(0.01943463, 0.0000001)

In [ ]:
# p_disease

In [ ]:
df_disease_state_pm_table = pd.DataFrame(
    [
        ['T',  p_disease_present], 
        ['F', 1-p_disease_present], 
    ],columns=['disease_present','p'])
df_disease_state_pm_table

In [ ]:
df_test_when_disease_cpm_table = pd.DataFrame(
    [
        ['T', 'T', p_test_positive_given_disease_present],
        ['T', 'F', 1- p_test_positive_given_disease_present],
        ['F', 'T', p_test_positive_given_disease_absent],
        ['F', 'F', 1- p_test_positive_given_disease_absent],
    ],columns=['disease_present','test_result','p']
)
df_test_when_disease_cpm_table

In [ ]:
import qgrid,itertools

* [pd.Categorical](https://pandas.pydata.org/pandas-docs/version/0.22.0/categorical.html)

In [ ]:
# disease_present = pd.Categorical(['F', 'T'], categories=['F', 'T'], ordered=True)
disease_present = pd.Categorical([], categories=['F', 'T'], ordered=True)
disease_present

In [ ]:
test_result = pd.Categorical([], categories=['F', 'T'], ordered=True)
test_result

In [ ]:
dp_tr_product = list(itertools.product(list(disease_present.categories),list(test_result.categories)))
dp_tr_product

In [ ]:
df_tmp = pd.DataFrame(dp_tr_product, columns=['disease_present', 'test_result'])
df_tmp['p'] = np.nan
qgrid_widget = qgrid.show_grid(df_tmp, show_toolbar=True)
qgrid_widget

In [ ]:
qgrid_widget.get_changed_df().head()

In [ ]:
# df_test_when_disease_cpm_table.pivot_table(index   = ['disease-present'], 
#                                            columns = ['test-result'],
#                                            values  = 'p')
df_test_when_disease_cpm_table.pivot_table(index   = list(df_test_when_disease_cpm_table.columns[:-2]), 
                                           columns = list(df_test_when_disease_cpm_table.columns[-2:-1]),
                                           values  = df_test_when_disease_cpm_table.columns[-1])
# df_test_when_disease_cpm_table.columns[-1]

* [networkx](https://github.com/networkx/networkx)
* [tutorial](https://networkx.github.io/documentation/stable/tutorial.html)

In [ ]:
import networkx as nx

In [ ]:
dg = nx.DiGraph()
# G.add_node(1)
dg.add_nodes_from(list(df_test_when_disease_cpm_table.columns[:-1]))
dg.add_edges_from([
    list(df_test_when_disease_cpm_table.columns[:-1]),
])

In [ ]:
list(nx.connected_components(dg.to_undirected()))

In [ ]:
nx.draw(dg, with_labels=True)

In [ ]:
# pos = nx.nx_agraph.graphviz_layout(dg)
# nx.draw(dg, with_labels=True, pos=pos)

In [ ]:
list(dg.nodes())

In [ ]:
list(dg.edges())

* [graphviz](https://github.com/xflr6/graphviz)
* [graphviz.readthedocs](https://graphviz.readthedocs.io/en/stable/)
* [examples/notebook.ipynb](https://github.com/xflr6/graphviz/blob/master/examples/notebook.ipynb)

In [ ]:
import graphviz

In [ ]:
dot = graphviz.Digraph(comment='The Round Table')

In [ ]:
dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

In [ ]:
dot

In [ ]:
dg_dot = graphviz.Digraph(comment='disease-network')
for node in dg.nodes():
    dg_dot.node(node)

for edge in dg.edges():
    dg_dot.edge(edge[0],edge[1])

dg_dot

* [xarray](https://xarray.pydata.org/en/stable/)
* [xArray_seminar](http://meteo.unican.es/work/xarray_seminar/xArray_seminar.html)

In [ ]:
import xarray as xr

In [ ]:
df_test_when_disease_cpm_table

In [ ]:
df_test_when_disease_cpm_xr = xr.DataArray(
    [
        [0.95, 0.05],
        [0.01, 0.99],
    ], 
    coords={'disease_present': ['F', 'T'], 'test_result': ['F', 'T']}, 
    dims=('disease_present', 'test_result'))
df_test_when_disease_cpm_xr

In [ ]:
df_test_when_disease_cpm_xr.loc['F', 'T']

In [ ]:
df_test_when_disease_cpm_xr.loc[{'disease_present': 'F', 'test_result': 'T'}]

In [ ]:
df_test_when_disease_cpm_xr.loc[{'disease_present': 'F', 'test_result': 'F'}]

In [ ]:
df_test_when_disease_cpm_xr.loc[{'disease_present': 'T', 'test_result': 'T'}]

In [ ]:
df_test_when_disease_cpm_xr.loc[{'disease_present': 'T', 'test_result': 'F'}]

In [ ]:
df_test_when_disease_cpm_xr.to_dataframe(name='p').reset_index()

In [ ]:
df_test_when_disease_cpm_table.to_xarray()

In [ ]:
pvalue_array = df_test_when_disease_cpm_table.groupby(['disease_present', 'test_result']).max().to_xarray().p.values
pvalue_array

In [ ]:
df_test_when_disease_cpm_xr

In [ ]:
%load_ext rpy2.ipython

* R [gl](https://www.rdocumentation.org/packages/base/versions/3.5.0/topics/gl) Generate Factor Levels

In [ ]:
%%R
dni3 <- dimnames(iris3)
print(dni3)
ii <- data.frame(matrix(aperm(iris3, c(1,3,2)), 
                        ncol = 4, 
                        dimnames = list(NULL, sub(" L.",".Length", sub(" W.",".Width", dni3[[2]])))),
                 Species = gl(3, 50, labels = sub("S", "s", sub("V", "v", dni3[[3]]))))
print(typeof(ii))
all.equal(ii, iris) # TRUE

In [ ]:
# import os
# os.environ['EDITOR']

In [ ]:
# %%R
# print(?lm)

In [ ]:
# %%R
# iris3

In [ ]:
# %%R
# iris

In [ ]:
# %%R
# aperm(iris3, c(1,3,2))

In [ ]:
# %%R
# sub(" L.",".Length", sub(" W.",".Width", dni3[[2]]))

In [ ]:
# %%R
# list(NULL, sub(" L.",".Length", sub(" W.",".Width", dni3[[2]])))

In [ ]:
# %%R
# matrix(aperm(iris3, c(1,3,2)),
#        ncol = 4,
#        dimnames = list(NULL, sub(" L.",".Length", sub(" W.",".Width", dni3[[2]]))))

In [ ]:
%%R -i df_test_when_disease_cpm_table
# print(typeof(df_test_when_disease_cpm_table))
# print(typeof(df_test_when_disease_cpm_table[1]))
# print(df_test_when_disease_cpm_table[1])
# print(df_test_when_disease_cpm_table[2])
df_test_when_disease_cpm_table

In [ ]:
%%R -i pvalue_array
pvalue_array

* [ENH: Mosaic plot and DataArray](https://github.com/pydata/xarray/issues/779)

In [ ]:
# %%R -i df_test_when_disease_cpm_xr
# df_test_when_disease_cpm_xr

In [ ]:
# cptSprinkler = matrix(c(0.4, 0.6, 0.01, 0.99), ncol = 2, dimnames = list("RAIN" = c("F","T"), "SPRINKLER" = c("T", "F")), byrow = TRUE)
# cptSprinkler

# cptGrassWet = array(c(0.0, 1.0, 0.8, 0.2, 0.9, 0.1, 0.99, 0.01), dim=c(2, 2, 2))
# cptGrassWet = aperm(cptGrassWet, perm=c(3,2,1))
# dimnames(cptGrassWet) = list("GRASSWET" = c("T", "F"), "RAIN" =  c("F", "T"), "SPRINKLER" = c("F", "T"))


In [ ]:
rpy2.robjects.r['pi']

In [ ]:
v = rpy2.robjects.FloatVector([1.1, 2.2, 3.3, 4.4, 5.5, 6.6])
m = rpy2.robjects.r['matrix'](v, nrow = 2)
rpy2.robjects.globalenv["mR"] = m
m

In [ ]:
%%R
mR

In [ ]:
df_test_when_disease_cpm_table

In [ ]:
def cpt_to_r(lcpm_df):
    lcpd_ds = lcpm_df.groupby(list(lcpm_df.columns[:-1])).max() # .to_xarray().p.values
    lcpd_xds = lcpd_ds.to_xarray()
    var_name = list(lcpd_xds.data_vars.keys())[0]
    lcpd_xr = lcpd_xds[var_name]
    
    # the array itself:
    nparray = np.array(lcpd_xr)

    # the dim shape
    dim_shape = lcpd_xr.shape
    rdim_shape = rpy2.robjects.IntVector(dim_shape)
    
    # the dimnames
    dim_name_values_pair = {}
    dim_names = lcpd_xr.dims
    for dim_name in dim_names:
        dim = lcpd_xr.coords[dim_name]
        dim_values = [str(dv) for dv in dim.values]
        dim_name_values_pair.update({dim_name: rpy2.robjects.StrVector(dim_values)})
    
    # now create the R array
    rarray = rpy2.robjects.r['array'](
        nparray, 
        dim=rdim_shape, 
        dimnames=rpy2.robjects.r['list'](**dim_name_values_pair)
    )
    return rarray

In [ ]:
dtcpm_test_when_disease = dsbasics.bn.DiscreteTabularCPM(df_test_when_disease_cpm_table)
dtcpm_test_when_disease

In [ ]:
dtcpm_test_when_disease.lcpm_xr.dims

In [ ]:
dtcpm_test_when_disease.rarray

In [ ]:
dtcpm_test_when_disease.model_string

In [ ]:
dtcpm_test_when_disease_nt = dsbasics.bn.DiscreteTabularCPM(df_test_when_disease_cpm_table, transform=False)
dtcpm_test_when_disease_nt

In [ ]:
dtcpm_test_when_disease_nt.lcpm_xr.dims

In [ ]:
rpy2.robjects.globalenv["dtcpm_test_when_disease_array"] = dtcpm_test_when_disease.rarray
rpy2.robjects.globalenv["dtcpm_test_when_disease_array_nt"] = dtcpm_test_when_disease_nt.rarray

In [ ]:
%%R 
print(dtcpm_test_when_disease_array)
print(dtcpm_test_when_disease_array_nt)

In [ ]:
dbn_disease = dsbasics.bn.DiscreteBayesNetwork([df_test_when_disease_cpm_table, df_disease_state_pm_table])

In [ ]:
dbn_disease.model_string

In [ ]:
print(dbn_disease.rnet)

In [ ]:
rpy2.robjects.globalenv["mR"] = cpt_to_r(df_test_when_disease_cpm_table)
rpy2.robjects.globalenv["mR"]

In [ ]:
%%R
t(mR)

In [ ]:
rpy2.robjects.globalenv["mR2"] = cpt_to_r(df_disease_state_pm_table)
rpy2.robjects.globalenv["mR2"]

In [ ]:
%%R
mR2

In [ ]:
%%R
net = model2network("[disease_present][test_result|disease_present]")
#net = model2network("[RAIN][SPRINKLER|RAIN]")
net

In [ ]:
%%R
dfit = custom.fit(net, dist = list("disease_present" = mR2, "test_result" = t(mR)))
dfit

In [ ]:
%%R
jtree = compile(as.grain(dfit))
jtree

In [ ]:
%%R
querygrain(jtree, nodes=c("disease_present", "test_result"), type="marginal")

In [ ]:
%%R
jprop = setFinding(jtree, nodes = "test_result", state="T")
jprop

In [ ]:
%%R
querygrain(jprop, nodes=c("disease_present"))#$GRASSWET
#jprop

In [ ]:
%%R
new.cpt = matrix(c(0.1, 0.2, 0.3, 0.2, 0.5, 0.6, 0.7, 0.3, 0.1), byrow = TRUE, ncol = 3, dimnames = list(B = c("a", "b", "c"), A = c("a", "b", "c")))
new.cpt

In [ ]:
%%R
as.table(new.cpt)

* [memory-layout-of-multi-dimensional-arrays](https://eli.thegreenplace.net/2015/memory-layout-of-multi-dimensional-arrays)
* [Arrays in R and Python](https://rstudio.github.io/reticulate/articles/arrays.html)
* numpy [reshape](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) documentation and its 'C' and 'F' parameter

In [ ]:
np_order_test = np.reshape(np.array(range(8)), (2,2,2), order='C') # order = 'C' vs. 'F'
np_order_test

In [ ]:
rpy2.robjects.globalenv["np_order_test"] = np_order_test

In [ ]:
%%R
print(np_order_test)
print(array(c(0:7),dim=c(2,2,2)))

In [ ]:
np_order_test2 = np.reshape(np_order_test.reshape(-1), np_order_test.shape, order='F')
np_order_test2

In [ ]:
rpy2.robjects.globalenv["np_order_test2"] = np_order_test2

In [ ]:
%%R
print(np_order_test2)
print(array(c(0:7),dim=c(2,2,2)))

In [ ]:
np_order_test.transpose()

In [ ]:
rpy2.robjects.globalenv["np_order_test3"] = np_order_test.transpose()

In [ ]:
%%R
print(np_order_test3)
print(array(c(0:7),dim=c(2,2,2)))

In [ ]:
xr_order_test = xr.DataArray(np_order_test, coords={'a': ['F', 'T'], 'b': ['f', 't'], 'c': ['y', 'n']}, dims=('a','b','c'))
xr_order_test

In [ ]:
xr_order_test.transpose(*('c','a','b'))

In [ ]:
old_dims = xr_order_test.dims
new_dims = old_dims[-1:] + old_dims[:-1]
new_dims
xr_order_test.transpose(*new_dims)

In [ ]:
xr_order_test

In [ ]:
tmp = np.reshape(np_order_test.reshape(-1),np_order_test.shape,order='F')
xr_order_test2 = xr.DataArray(tmp, coords={'a': ['F', 'T'], 'b': ['f', 't'], 'c': ['y', 'n']}, dims=('a','b','c'))
xr_order_test2

In [ ]:
old_dims = xr_order_test2.dims
new_dims = old_dims[-1:] + old_dims[:-1]
new_dims
xr_order_test3 = xr_order_test2.transpose(*new_dims)
xr_order_test3

In [ ]:
rpy2.robjects.globalenv["xr_order_test3"] = xr_order_test3.values

In [ ]:
%%R
print(xr_order_test3)
#print(array(c(0:7),dim=c(2,2,2)))
print(aperm(array(c(0:7),dim=c(2,2,2)),c(3,1,2)))

In [ ]:
xr_order_test.loc[dict(a='F',b='f',c='n')]

In [ ]:
xr_order_test2.loc[dict(a='F',b='f',c='n')]

In [ ]:
xr_order_test3.loc[dict(a='F',b='f',c='n')]

![rain-sprinkler-grass](https://i.stack.imgur.com/yc2qx.png)

In [ ]:
df_rain_pm = pd.DataFrame(
    [
        ['T', 0.2],
        ['F', 0.8],
    ],columns=['rain','p']
)
df_rain_pm

In [ ]:
df_sprinkler_cpm = pd.DataFrame(
    [
        ['F','F', 0.6],
        ['F','T', 0.4],
        ['T','F', 0.99],
        ['T','T', 0.01],
    ],columns=['rain','sprinkler','p']
)
df_sprinkler_cpm

In [ ]:
df_grasswet_cpm = pd.DataFrame(
    [
        ['F','F','F',1.0],
        ['F','F','T',0.0],
        ['F','T','F',0.2],
        ['F','T','T',0.8],

        ['T','F','F',0.1],
        ['T','F','T',0.9],
        ['T','T','F',0.01],
        ['T','T','T',0.99],
    ],columns=['sprinkler','rain','grasswet','p']
)
df_grasswet_cpm

In [ ]:
df_grasswet_cpm.pivot_table(index=['rain','sprinkler'],columns=['grasswet'],values=['p']).reset_index()

In [ ]:
dtcpm_grasswet    = dsbasics.bn.DiscreteTabularCPM(df_grasswet_cpm)
dtcpm_grasswet_nt = dsbasics.bn.DiscreteTabularCPM(df_grasswet_cpm,transform=False)
dtcpm_grasswet

In [ ]:
list(dtcpm_grasswet.parents)

In [ ]:
dtcpm_grasswet.to_cpm_table()

In [ ]:
dtcpm_grasswet.lcpm_xr.dims

In [ ]:
dtcpm_grasswet.lcpm_xr.loc[dict(rain='F',sprinkler='T')]

In [ ]:
dtcpm_grasswet.model_string

In [ ]:
rpy2.robjects.globalenv["dtcpm_grasswet_array"] = dtcpm_grasswet.rarray
rpy2.robjects.globalenv["dtcpm_grasswet_array_nt"] = dtcpm_grasswet_nt.rarray

In [ ]:
%%R
print(dtcpm_grasswet_array)
print(dtcpm_grasswet_array_nt)

In [ ]:
dbn_rain_springkler_grasswet = dsbasics.bn.DiscreteBayesNetwork([df_rain_pm, df_sprinkler_cpm, df_grasswet_cpm])

In [ ]:
dbn_rain_springkler_grasswet.model_string

In [ ]:
print(dbn_rain_springkler_grasswet.rnet)

In [ ]:
print(dbn_rain_springkler_grasswet.rfit)

In [ ]:
dtcpm_grasswet.to_cpm_table()

In [ ]:
evidence = dict(grasswet='T')
evidence

In [ ]:
dbn_rain_springkler_grasswet = dsbasics.bn.DiscreteBayesNetwork([df_rain_pm, df_sprinkler_cpm, df_grasswet_cpm])

In [ ]:
result = dbn_rain_springkler_grasswet.exact_query(evidence, ['rain','sprinkler'])

In [ ]:
result

In [ ]:
dbn_rain_springkler_grasswet.exact_query(dict(grasswet='T',spinkler='F'), ['rain'])